In [3]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

In [6]:
data = pd.read_csv("D:/1mini project/internship/SPAM SMS DETECTION/spam.csv", encoding='latin-1')

data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:

data = data[['v2', 'v1']]

data = data.rename(columns={'v1': 'label', 'v2': 'messages'})

data.head()

,messages,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [8]:
data.isnull().sum()

messages    0
label       0
dtype: int64

In [9]:
data.duplicated().sum()
data=data.drop_duplicates(keep='first')

403

In [11]:
data.shape

(5169, 2)

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [19]:
STOPWORDS=set(stopwords.words('english'))

def clean_text(text):
    text=text.lower()
    text=re.sub(r'[^0-9a-zA-Z]',' ',text)
    text=re.sub(r'\s+',' ',text)
    text=" ".join(word for word in text.split() if word not in STOPWORDS)
    return text

In [20]:
data['clean_text']=data['messages'].apply(clean_text)
data.head()

,messages,label,clean_text
0,"Go until jurong point, crazy.. Available only ...",ham,go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,ham,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah think goes usf lives around though


In [21]:
X=data['clean_text']
y=data['label']

In [22]:
data['label'] = data['label'].map({'ham': 0, 'spam': 1})

In [23]:
data.head()

,messages,label,clean_text
0,"Go until jurong point, crazy.. Available only ...",0,go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,0,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,0,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",0,nah think goes usf lives around though


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def classify(model, X, y, epochs=10):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)

    pipeline_model = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', model)
    ])

    if hasattr(pipeline_model.named_steps['clf'], 'epochs'):
        pipeline_model.named_steps['clf'].epochs = epochs
    
    pipeline_model.fit(X_train, y_train)
    y_pred = pipeline_model.predict(X_test)
    
    print('Number of epochs:', epochs)
    print('Accuracy:', pipeline_model.score(X_test, y_test) * 100)
    print(classification_report(y_test, y_pred))
    
    return pipeline_model

In [26]:
from sklearn.linear_model import LogisticRegression
model_lg=LogisticRegression(C=90, max_iter=100,penalty='l2')
epochs=15
classify(model_lg,X,y,epochs)

Number of epochs: 15
Accuracy: 98.06651198762569
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1130
        spam       0.99      0.86      0.92       163

    accuracy                           0.98      1293
   macro avg       0.98      0.93      0.95      1293
weighted avg       0.98      0.98      0.98      1293



Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=90))])